In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
batch_size = 128
RELU_nodes = 1024

def hidden_layer1(dataset,weights1,weights2,biases1,biases2):
    layer_1 = tf.matmul(dataset, weights1) + biases1
    layer_1 = tf.nn.relu(layer_1)
    logits = tf.matmul(layer_1, weights2) + biases2
    return logits


    
graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
#Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
            shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, RELU_nodes ],seed=13))
    biases1 = tf.Variable(tf.zeros([RELU_nodes]))
    
    weights2 = tf.Variable(
        tf.truncated_normal([RELU_nodes, num_labels],seed=713))
    biases2 = tf.Variable(tf.zeros([num_labels]))  
  
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = hidden_layer1(tf_train_dataset,weights1,weights2,biases1,biases2)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    logits_val =  hidden_layer1(tf_valid_dataset,weights1,weights2,biases1,biases2)
    logits_test =  hidden_layer1(tf_test_dataset,weights1,weights2,biases1,biases2)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(logits_val)
    test_prediction = tf.nn.softmax(logits_test)

In [7]:
num_steps = 5001

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
        / predictions.shape[0])

with tf.Session(graph=graph) as session:
# This is a one-time operation which ensures the parameters get initialized as
# we described in the graph: random weights for the matrix, zeros for the
# biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 269.878601
Minibatch accuracy: 5.5%
Validation accuracy: 37.4%
Minibatch loss at step 500: 15.367887
Minibatch accuracy: 78.1%
Validation accuracy: 76.9%
Minibatch loss at step 1000: 10.003668
Minibatch accuracy: 79.7%
Validation accuracy: 78.9%
Minibatch loss at step 1500: 6.182476
Minibatch accuracy: 78.9%
Validation accuracy: 81.2%
Minibatch loss at step 2000: 11.827445
Minibatch accuracy: 80.5%
Validation accuracy: 79.7%
Minibatch loss at step 2500: 6.020215
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
Minibatch loss at step 3000: 4.542219
Minibatch accuracy: 81.2%
Validation accuracy: 78.8%
Minibatch loss at step 3500: 4.372362
Minibatch accuracy: 85.9%
Validation accuracy: 80.0%
Minibatch loss at step 4000: 3.722557
Minibatch accuracy: 87.5%
Validation accuracy: 82.2%
Minibatch loss at step 4500: 1.547595
Minibatch accuracy: 88.3%
Validation accuracy: 81.4%
Minibatch loss at step 5000: 0.753433
Minibatch accuracy: 89.8%
Validation acc